Im quite scared of loading this whole thing onto my computer


In [2]:
from datasets import load_dataset
import tiktoken
from pathlib import Path
import os
import torch
import random

/Users/sydneysim/Documents/codingProjects/GPT2-exploration/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("openwebtext")

DataFilesNotFoundError: No (supported) data files found in openwebtext

In [5]:
train_data=dataset['train']

split the big dataset here into many text files, will incorporate it into the text data loader at a later time

In [3]:
enc=tiktoken.get_encoding('gpt2')
tok_limit=10000000 #think about it 10 million tokens means ten mil line array, which could be i/o intensive?
output_dir=Path("../data/openwebtext")
output_dir.mkdir(exist_ok=True)

buffer = []
file_idx = 0
token_count = 0

for example in train_data:
    text = example["text"]
    tokens = enc.encode(text)
    len_token = len(tokens)

    if token_count + len_token >= tok_limit:
        output_path = output_dir / f"tokens_part_{file_idx}.pt"
        torch.save(torch.tensor(buffer, dtype=torch.long), output_path)

        file_idx += 1
        buffer = []
        token_count = 0

    buffer.extend(tokens)
    token_count += len_token

if buffer:
    output_path = output_dir / f"tokens_part_{file_idx}.pt"
    torch.save(torch.tensor(buffer, dtype=torch.long), output_path)
    


NameError: name 'train_data' is not defined

In [25]:
import os, os.path

# simple version for working with CWD
len([name for name in os.listdir('../data/openwebtext/')])
dir_path=Path('../data/openwebtext/')

print(len(sorted(dir_path.glob("*.pt"))))


904


In [4]:
from torch.utils.data import IterableDataset, DataLoader, DistributedSampler

class DirStreamingDataset(IterableDataset):
    """
    openwebtext directory now contains pt files in binary, precomputed with the GPT 2 encodings

    Each file is 10 million tokens, total of 904 files
    """

    def __init__(self, path: str | os.PathLike, block_size: int):
        self.dir_path = Path(path)
        if not self.dir_path.exists():
            raise FileNotFoundError(path)
        
        self.token_files=sorted(self.dir_path.glob("*.pt"))
        if not self.token_files:
            raise ValueError(f"No .pt files found in {self.dir_path}")
        self.num_files=len(self.token_files)
        self.block_size = block_size
        self.num_token_per_file=10000000

        # introduce per-worker offset to avoid every worker seeing the
        # same sequence order when shuffling is disallowed.
        # DataLoader(worker_init_fn) will set this value.
        self._worker_offset = 0


    # DataLoader with an IterableDataset must NOT shuffle. We instead start each worker at a random offset to approximate randomness.
    def set_worker_offset(self, offset: int):
        self._worker_offset = offset

    def get_dataset_token_count(self):
        return self.num_files*self.num_token_per_file

    def __iter__(self):
        rng = torch.Generator().manual_seed(42 + self._worker_offset)
        file_indices = list(range(len(self.token_files)))
        random.shuffle(file_indices)

        while True:
            for idx in file_indices:
                tokens = torch.load(self.token_files[idx])
                n = len(tokens)
                pos = torch.randint(0, n - self.block_size - 1, (1,), generator=rng).item()

                while pos + self.block_size + 1 < n:
                    #now instead of randomising the start position, we randomise which file we see, then the start position is sequential
                    chunk = tokens[pos : pos + self.block_size + 1]
                    x = chunk[:-1]
                    y = chunk[1:]
                    pos += self.block_size
                    yield x, y# DataLoader → (B,T)

def make_loader(dataset: DirStreamingDataset, batch_size: int, num_workers: int,
                sampler: DistributedSampler | None, device_is_cuda: bool):
    def worker_init_fn(worker_id: int):
        worker_info = torch.utils.data.get_worker_info()
        dataset: DirStreamingDataset = worker_info.dataset
        offset = random.randint(0, len(dataset.tokens) - 1)
        dataset.set_worker_offset(offset)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=sampler,
        num_workers=num_workers,
        pin_memory=device_is_cuda,
        prefetch_factor=None, # does not run when 0 num_workers
        persistent_workers=(num_workers > 0),
        worker_init_fn=None,
    )


In [75]:
data_path="../data/openwebtext"
B, T        = 64, 1024
lr          = 3e-4
max_steps   = 100
enc         = tiktoken.get_encoding("gpt2")
sampler=None

dataset = DirStreamingDataset(path=data_path, block_size=T)
loader = make_loader(
        dataset,
        batch_size=B,
        num_workers=0,            # no separate workers ➜ no pickling
        sampler=sampler,
        device_is_cuda=False,
    )

for step, (x, y) in enumerate(loader):
    if step >= max_steps:
        break

    print(enc.decode(x[0].tolist()))
    print(x)
    print("---")
    print(y)

Valentin Moldavsky vs. Carl Seumanutafa

Jordan Howard vs. Johnny Marigo

Andre Fialho vs. A.J. Matthews

Amanda Bell vs. Brittney Elkin

Kemmyelle Haley vs. Logan Storley

William Florentino vs. Jonathan Gary

E.J. Brooks vs. Guilherme Vasconcelos

Katy Collins vs. Bruna Vargas

Guillermo Gonzalez vs. Fernando Trevino

For more on Bellator 181, check out the MMA Rumors section of the site.According to a report from Tim McManus of Birds 24/7, the Philadelphia Eagles have hired former ACC senior replay official and Northeast Philadelphia resident Frank Kosman to the team's staff. McManus notes that Kosman has been added to help Eagles head coach Chip Kelly whether he should challenge plays or not.

Eagles offensive coordinator Pat Shurmur, who attends games in the coaches' booth, was previously responsible for these duties. The Eagles were only successful on three out of seven challenge attempts in 2013. Clearly, there was room for improvement.

Adding a former replay official seems lik

In [8]:
def smoke_test(loader, 
               B:int,
               T:int,
               n_batches: int = 3,
               ):
        print("running loader smoke-test:")
        for step, (x, y) in enumerate(loader):
            print(f"\nBatch {step}  ------------------------------------")
            print(f"x shape {tuple(x.shape)}, dtype {x.dtype}")
            print(f"y shape {tuple(y.shape)}, dtype {y.dtype}")

            # shape & dtype checks
            assert x.shape == (B, T)
            assert y.shape == (B, T)
            assert x.dtype == torch.long and y.dtype == torch.long

            # target-shift check:  y[b, t]  must equal  x[b, t+1]
            same = torch.all(y[:, :-1] == x[:, 1:])
            if not same:
                idx = (y[:, :-1] != x[:, 1:]).nonzero(as_tuple=False)[0]
                b, t = idx.tolist()
                raise ValueError(f"y is not a left-shifted copy of x at (batch={b}, pos={t})")
            print("✓ shapes & left-shift OK")

            # simple uniqueness check – tokens should not all be identical
            uniq = torch.unique(x).numel()
            print(f"unique tokens in x: {uniq}")
            assert uniq > 1, "looks like the same token repeated – did shuffling break?"

            if step + 1 == n_batches:
                break

        print("\n**** DataLoader smoke-test passed ****")

smoke_test(loader, 64, 1024)

running loader smoke-test:

Batch 0  ------------------------------------
x shape (64, 1024), dtype torch.int64
y shape (64, 1024), dtype torch.int64
✓ shapes & left-shift OK
unique tokens in x: 10940

Batch 1  ------------------------------------
x shape (64, 1024), dtype torch.int64
y shape (64, 1024), dtype torch.int64
✓ shapes & left-shift OK
unique tokens in x: 10241

Batch 2  ------------------------------------
x shape (64, 1024), dtype torch.int64
y shape (64, 1024), dtype torch.int64
✓ shapes & left-shift OK
unique tokens in x: 9980

**** DataLoader smoke-test passed ****
